# Extensions

The Polya-Gamma Logistic Regression is convenient for its conjugacy. However, for non-simulated applications, there is no generally good setting for the mean and covariance parameters. The Bayesian way to handle this issue is to place priors on both. In this section, we'll extend the Polya-Gamma algorithm by incorporating hyperpriors on the coefficient mean and variance parameters. For most of this section, we'll draw from the conjugate derivations given by Murphy in: https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf.

To start, we'll move away from the notation in the PG-TS paper to the more common notation:

-   from: $p(\theta) \sim MVN(\mathbf{b}, \mathbf{B})$
-   to: $p(\beta) \sim MVN(\mu, \Sigma)$

We'll also work with precision matrices in some cases. The precision matrix is simply $\Lambda = \Sigma^{-1}$.

## Restate Polya-Gamma Logistic Regression basics

To restate, the conditional posterior of $\beta$ given latent variables $\mathbf{\omega} = [\omega_1, \ldots, \omega_t]$ and past rewards $\mathbf{r} = [r_1, \ldots, r_t]$ up to time step $t$ is a Gaussian:

\begin{align}
p(\beta | \mathbf{r})
    &= p(\beta) \prod_{i=1}^t L_i(r_i | \beta, \mathbf{x}_i)  \\
    &= p(\beta) \prod_{i=1}^t L_i(z_i | \omega_i, \beta, \mathbf{x}_i) P(\omega_i | \beta, \mathbf{x}_i)  \\
    &\propto p(\beta) \ exp \left\{ -\frac{1}{2} (z - \mathbf{X}\beta)^T \mathbf{\Omega} (z - \mathbf{X}\theta) \right\},
\end{align}

where $z = (\kappa_1 / \omega_1, \ldots, \kappa_t / \omega_t)$, $\mathbf{\kappa} = [r_1 - 1/2, \ldots, r_t - 1/2]$, and $\mathbf{\Omega} = diag(\omega_1, \ldots, \omega_t)$. This is a conditionally Gaussian likelihood in $\theta$, with working responses $z$, design matrix $\mathbf{X}$, and diagonal covariance matrix $\mathbf{\Omega}^{-1}$. With a multivariate Gaussian prior for $\beta \sim MVN(\mu, \Sigma)$:

$$p(\beta) \propto exp\left\{ -\frac{1}{2} (\beta - \mu)^T \Sigma^{-1} (\beta - \mu) \right\},$$

this identity leads to an efficient Gibbs sampler. The main parameters are drawn from a Gaussian distribution, which is parameterized by latent variables drawn from the PG distribution. The two steps are:

\begin{align}
\omega_i | \beta, \mathbf{x}_i &\sim PG(1, \mathbf{x}_i^T \beta)
    &\mathbf{V}_\omega &= (\mathbf{X}^T \mathbf{\Omega} \mathbf{X} + \Lambda)^{-1}  \\
\beta | \mathbf{\omega}, \mathbf{r}, \mathbf{X} &\sim N(\mathbf{m}_\omega, \mathbf{V}_\omega)
    &\mathbf{m}_\omega &= \mathbf{V}_\omega (\mathbf{X}^T \mathbf{\kappa} + \Lambda\mu)
\end{align}

The PGM for this is shown below:

<img src="images/pg-logistic-response-model-pgm.png"
     style="margin-left: auto; margin-right: auto; display: block; height: 30%; width: 30%"></img>

Next let's introduce a set of hyperparameters $\Phi_0$ for the hyperprior over the coefficient prior mean and covariance $h(\mu, \Sigma | \Phi_0)$:

$$p(\beta | \Phi_0) \propto \mathcal{N}(\beta | \mu, \Sigma) h(\mu, \Sigma | \Phi_0)$$

The following sections will investigate various options for $h$ and $\Phi_0$.

## Normal Inverse Wishart

The natural conjugate prior for the multivariate normal (parameterized in terms of mean vector and precision matrix) is the Normal Inverse-Wishart (NIW):

\begin{align}
\Sigma &\sim IW{\nu_0}(\Lambda_0^{-1})  \\
\mu | \Sigma &\sim \mathcal{N}(\mu_0, \Sigma / \kappa_0)  \\
p(\mu, \Sigma) &\overset{\text{def}}{=} NIW(\mu_0, \kappa_0, \Lambda_0, \nu_0)
\end{align}

With this hyperprior $h = NIW$, the posterior given $\beta$ at time $t$ and the hyperparameters $\Phi_0 = \{\kappa_0, \mu_0, \nu_0, \Lambda_0\}$ is:

\begin{align}
p(\mu, \Sigma | \beta, \Phi)
    &\propto \mathcal{N}(\beta | \mu, \Sigma) NWi(\mu, \Sigma | \Phi_0)  \\
    &= NIW(\mu, \Sigma | \mu_t, \kappa_t, \Lambda_t, \nu_t)  \\
\kappa_t &= \kappa_0 + d  \\
\mu_t &= \frac{\kappa_0 \mu_0 + d \bar{\beta}}{\kappa_t}  \\
\nu_t &= \nu_0 + d  \\
\Lambda_t &= \Lambda_0 + S + \frac{\kappa_0 d}{\kappa_t} (\bar{\beta} - \mu_0)(\bar{\beta} - \mu_0)^T,
\end{align}

where $d$ is the number of predictors (and thus the dimension of $\beta$) and $S$ is the matrix of sum of squares (scatter matrix):

$$S = \sum_{i=1}^p (\beta_i - \bar{\beta})(\beta_i - \bar{\beta})^T$$

We'll call the set of updated hyperparameters $\Phi_t$.

The PGM for this model is shown below:

<img src="images/bayes-pg-logistic-response-model-pgm.png"
     style="margin-left: auto; margin-right: auto; display: block; height: 30%; width: 30%"></img>

This model is still semi-conjugate, so we can build an efficient Gibbs sampler to draw $S$ samples from the posterior:

1.  **Initialize**:
    -   $\Sigma^{(0)} \sim IW_{\nu_0}(\Lambda_0^{-1})$,
    -   $\mu^{(0)} \sim \mathcal{N}(\mu_0, \Sigma^{(0)} / \kappa_0)$,
    -   $\beta^{(0)} \sim \mathcal{N}(\mu^{(0)}, \Sigma^{(0)})$
2.  **Precompute**: values that won't change:
    -   $\kappa_t = \kappa_0 + d$
    -   $\nu_t = \nu_0 + d$
2.  **Sample**: for $s = 1, \ldots, S$:
    -   for $i = 1, \ldots, t$
        -   $\omega_i^{(s)} \sim PG(1, \mathbf{x}_i^T \beta^{(s - 1)})$
    -   $\beta^{(s)} \sim \mathcal{N}(\mathbf{m}_\omega, \mathbf{V}_\omega)$
        -   $\mathbf{V}_\omega = (\mathbf{X}^T \mathbf{\Omega} \mathbf{X} + \Sigma^{(s - 1)^{-1}})^{-1}$
        -   $\mathbf{m}_\omega = \mathbf{V}_\omega (\mathbf{X}^T \kappa + \Sigma^{(s - 1)^{-1}} \mu^{(s - 1)})$
    -   $\mu^{(s)}, \Sigma^{(s)} \sim NIW(\kappa_t, \mu_t, \nu_t, \Lambda_t)$
        -   $\mu_t = (\kappa_0 \mu_0 + d \bar{\beta}^{(s)}) / \kappa_t$
        -   $\Lambda_t = \Lambda_0 + S^{(s)} + \frac{\kappa_0 d}{\kappa_t} (\bar{\beta}^{(s)} - \mu_0)(\bar{\beta}^{(s)} - \mu_0)^T$